# Setup Enviroments 

### Download Request library

**NOTE**.  Only download this on initial start of platform

In [1]:
# Only execute on initial start of platform
!pip install requests

### Define Spotify Developer Credentials

In [2]:
client_id = "cd1845f23d914228b14f6bed139ee594"
client_secret = ""

# Create an API Client Class for Spotify, `SpotifyAPI`

Rather than creating a function for Spotify Access Token that's a state-like item (change overtime), let's create a Class that can response to changes overtime.

This class authorize your access to Spotify API by: 
  1. Gain credentials by passing in Spotify Dev. ID and Secret Key
  1. Gain authorization by getting token
  

_NOTE_
    - Request Authorization's token uses "Basic" within the authorization header string.
    - While using the access token to access Spotify Web API uses "Bearer" within the authorization header string.
    - REF. https://developer.spotify.com/documentation/general/guides/authorization-guide/#client-credentials-flow

In [7]:
import requests
import base64
import datetime
from urllib.parse import urlencode

class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    
    token_URL = "https://accounts.spotify.com/api/token"
    
    
    
    
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    
    
    
    
    def get_client_credentials(self):
        """
        METHOD: Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if (client_id == None) or (client_secret == None):
            raise Exception("You must set a Client ID or a Client Secret with Spotify Developer Account.")
        
        # Convert Spotify Credentials from string to base64-bytes (decoded)
        client_creds = f"{client_id}:{client_secret}"                # type: String
        client_creds_b64 = base64.b64encode(client_creds.encode())   # type: Base64-Byte
        return client_creds_b64.decode()

    
    
    
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"             # Basic <base64 encoded client_id:client_secret>
        }
        
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        }
        
    
    
    
    
    def perform_auth(self):
        """
        METHOD: Authenticate to Spotify by acquiring token
        """
        token_URL = self.token_URL
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        
        # Get Token Response
        response = requests.post(token_URL, data=token_data, headers=token_headers)

        
        # Exit if Request is Invalid...
        if response.status_code not in range (200, 299):
            raise Exception("Could not authenticate client.")
            # return False        
        
        
        # Else, proceed...
        token_response_data = response.json()

        access_token = token_response_data['access_token']

        expires_in = token_response_data['expires_in']            # Unit: Seconds
        now = datetime.datetime.now()
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now

        return True
    
    
    
    
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token
    
    
    
    
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
    
    
    
    
    
    def get_resource(self, lookup_id, resource_type="albums", version="v1"):
        headers = self.get_resource_header()
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        
        return r.json()
    
    
    
    
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type="albums")
    
    
    
    
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type="artists")
    
    
    
    
    
    def search(self, query, search_type='artist'):
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        data = urlencode({"q": query, "type": search_type.lower()})
        lookup_url = f"{endpoint}?{data}"
        
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):
            return {}

        return r.json()

# Calling API Client Class, `SpotifyAPI`

In [8]:
sp = SpotifyAPI(client_id, client_secret)

# Search Spotify

The `Search` section of Spotify API endpoint retrieves Spotify Catalog information about albums, artists, playlists, tracks, shows or episodes that match a keyword string.

**REF**: https://developer.spotify.com/documentation/web-api/reference/search/search/

In [11]:
sp.search("Something Holy", search_type="Track")

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Something+Holy&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/03uMw43UVu9MsQCcHVSGjX'},
       'href': 'https://api.spotify.com/v1/artists/03uMw43UVu9MsQCcHVSGjX',
       'id': '03uMw43UVu9MsQCcHVSGjX',
       'name': 'Alice Phoebe Lou',
       'type': 'artist',
       'uri': 'spotify:artist:03uMw43UVu9MsQCcHVSGjX'}],
     'available_markets': ['AD',
      'AE',
      'AL',
      'AR',
      'AT',
      'AU',
      'BA',
      'BE',
      'BG',
      'BH',
      'BO',
      'BR',
      'BY',
      'CA',
      'CH',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DE',
      'DK',
      'DO',
      'DZ',
      'EC',
      'EE',
      'EG',
      'ES',
      'FI',
      'FR',
      'GB',
      'GR',
      'GT',
      'HK',
      'HN',
      'HR',
      'HU',
      'ID',
      'IE',
      'IL',
      'I

In [10]:
sp.get_artist("03uMw43UVu9MsQCcHVSGjX")

{'external_urls': {'spotify': 'https://open.spotify.com/artist/03uMw43UVu9MsQCcHVSGjX'},
 'followers': {'href': None, 'total': 117757},
 'genres': ['south african alternative'],
 'href': 'https://api.spotify.com/v1/artists/03uMw43UVu9MsQCcHVSGjX',
 'id': '03uMw43UVu9MsQCcHVSGjX',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/07fbd93e50f41ac4f7c6750cee2ba5e26ea099fd',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ec61d8f88dd1237636b52a39c210b59f5e70353e',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/9ee18d57a40bee713161c18a13ed4ce36324ccf2',
   'width': 160}],
 'name': 'Alice Phoebe Lou',
 'popularity': 53,
 'type': 'artist',
 'uri': 'spotify:artist:03uMw43UVu9MsQCcHVSGjX'}

In [12]:
sp.get_album("4r8HtFE8TGFtqEpMV7mU60")

{'album_type': 'album',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/03uMw43UVu9MsQCcHVSGjX'},
   'href': 'https://api.spotify.com/v1/artists/03uMw43UVu9MsQCcHVSGjX',
   'id': '03uMw43UVu9MsQCcHVSGjX',
   'name': 'Alice Phoebe Lou',
   'type': 'artist',
   'uri': 'spotify:artist:03uMw43UVu9MsQCcHVSGjX'}],
 'available_markets': ['AD',
  'AE',
  'AL',
  'AR',
  'AT',
  'AU',
  'BA',
  'BE',
  'BG',
  'BH',
  'BO',
  'BR',
  'BY',
  'CA',
  'CH',
  'CL',
  'CO',
  'CR',
  'CY',
  'CZ',
  'DE',
  'DK',
  'DO',
  'DZ',
  'EC',
  'EE',
  'EG',
  'ES',
  'FI',
  'FR',
  'GB',
  'GR',
  'GT',
  'HK',
  'HN',
  'HR',
  'HU',
  'ID',
  'IE',
  'IL',
  'IN',
  'IS',
  'IT',
  'JO',
  'JP',
  'KW',
  'KZ',
  'LB',
  'LI',
  'LT',
  'LU',
  'LV',
  'MA',
  'MC',
  'MD',
  'ME',
  'MK',
  'MT',
  'MX',
  'MY',
  'NI',
  'NL',
  'NO',
  'NZ',
  'OM',
  'PA',
  'PE',
  'PH',
  'PL',
  'PS',
  'PT',
  'PY',
  'QA',
  'RO',
  'RS',
  'RU',
  'SA',
  'SE',
  'SG',
  'SI',
 

# REFERENCES
  - Spotify User's Credentials https://developer.spotify.com/documentation/general/guides/authorization-guide/#client-credentials-flow
  - Spotify Web API - Search https://developer.spotify.com/documentation/web-api/reference/search/search/
  - Spotify Web API - Artists https://developer.spotify.com/documentation/web-api/reference/artists/
  - Spotify Web API - Albums https://developer.spotify.com/documentation/web-api/reference/albums/